In [3]:
from minio import Minio
from minio.error import S3Error
import pandas as pd

In [4]:
# Initialize MinIO client
client = Minio('localhost:9000',
                     access_key='minioadmin',
                     secret_key='minioadmin',
                     secure=False)  # Change to True if using HTTPS

# Example: List buckets
try:
    buckets = client.list_buckets()
    for bucket in buckets:
        print(bucket.name)
except S3Error as e:
    print("Error:", e)

try:
    bucket_name = "ecommerce"
    objects = client.list_objects(bucket_name)
    for obj in objects:
        if obj.object_name.endswith('/'):
            print("Directory:", obj.object_name)
        else:
            print("File:", obj.object_name)
except S3Error as e:
    print("Error in listing objects:", e)

ecommerce
homework
File: 2020-Apr.csv
File: 2020-Feb.csv
File: 2020-Mar.csv


In [5]:
bucket_name = "ecommerce"  # Corrected bucket name
dec_data = "2019-Dec.csv"  # Corrected object name

try:
    response = client.get_object(bucket_name, dec_data)
    data = pd.read_csv(response)   # this data will be used in step 8
    print(data.head())  # Display first few rows
except S3Error as e:
    print("Error in reading object:", e)
finally:
    if 'response' in locals():
        response.close()
        response.release_conn()

Error in reading object: S3 operation failed; code: NoSuchKey, message: The specified key does not exist., resource: /ecommerce/2019-Dec.csv, request_id: 179F3E1E136AC807, host_id: dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8, bucket_name: ecommerce, object_name: 2019-Dec.csv


In [6]:
mar_data = "2020-Mar.csv"  # Corrected object name

try:
    response = client.get_object(bucket_name, mar_data)
    data = pd.read_csv(response)   # this data will be used in step 8
    print(data.head())  # Display first few rows
except S3Error as e:
    print("Error in reading object:", e)
finally:
    if 'response' in locals():
        response.close()
        response.release_conn()

In [ ]:
apr_data = "2020-Apr.csv"  # Corrected object name

try:
    response = client.get_object(bucket_name, apr_data)
    data = pd.read_csv(response)   # this data will be used in step 8
    print(data.head())  # Display first few rows
except S3Error as e:
    print("Error in reading object:", e)
finally:
    if 'response' in locals():
        response.close()
        response.release_conn()

                event_time event_type  product_id          category_id  \
0  2020-04-01 00:00:00 UTC       view     1201465  2232732101407408685   
1  2020-04-01 00:00:01 UTC       view     1307156  2053013554658804075   
2  2020-04-01 00:00:01 UTC       view     1480477  2053013563835941749   
3  2020-04-01 00:00:02 UTC       view     1307050  2053013554658804075   
4  2020-04-01 00:00:03 UTC       view     9500109  2232732104175649385   

                      category_code     brand    price    user_id  \
0             apparel.shoes.slipons   samsung   230.38  568984877   
1       electronics.audio.headphone     apple  1352.67  514955500   
2  appliances.kitchen.refrigerators     apple  1184.05  633645770   
3       electronics.audio.headphone     apple  1724.34  564933778   
4                     apparel.scarf  defender    25.05  530206135   

                           user_session  
0  e2456cef-2d4f-42b9-a53a-8893cb0c6851  
1  38f43134-de83-4710-ae0a-326677d292c6  
2  16aba270-b3

In [ ]:
feb_data = "2020-Feb.csv"  # Corrected object name

try:
    response = client.get_object(bucket_name, feb_data)
    data = pd.read_csv(response)   # this data will be used in step 8
    print(data.head())  # Display first few rows
except S3Error as e:
    print("Error in reading object:", e)
finally:
    if 'response' in locals():
        response.close()
        response.release_conn()

                event_time event_type  product_id          category_id  \
0  2020-02-01 00:00:00 UTC       view   100010426  2232732115290555244   
1  2020-02-01 00:00:00 UTC       view   100071077  2232732079706079299   
2  2020-02-01 00:00:00 UTC       view     1004665  2232732093077520756   
3  2020-02-01 00:00:01 UTC       view     1002544  2232732093077520756   
4  2020-02-01 00:00:01 UTC       view   100010151  2053013563835941749   

                      category_code    brand   price    user_id  \
0                     apparel.shoes      NaN   19.31  581521975   
1                     sport.bicycle    crown   11.88  596313635   
2          construction.tools.light  samsung  838.23  608821035   
3          construction.tools.light    apple  410.42  530488542   
4  appliances.kitchen.refrigerators   pulser  332.03  583795643   

                           user_session  
0  82d5664d-7980-43e1-ab27-a2f05e991760  
1  3df590c0-e667-4f5a-aa87-f47749cd6408  
2  3e271f43-db51-46bd-b215